# Export Relative & Normalized Trajectories (per keypoint folders)

- **Relative plots** (x,y relative to `tail set`) are written to `process/<keypoint>/`  
- **Normalized plots** (nx,ny) are written to **`process/<keypoint>_nor/`** ← (updated as requested)  
- Points are **not** drawn (lines only).  
- Axis ranges are configurable for both relative and normalized views.  
- No likelihood masking or interpolation is applied.


In [5]:
# ==== Settings (edit here) ====
IN_DIR   = r"C:\kanno\vscode\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\data\train\train_csv"          # DLC 3-level header CSV directory
OUT_DIR  = r"C:\kanno\vscode\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\process"                      # Output root (e.g., ...\\process)

KEYPOINTS = [  # must include "tail set" for relative plots
    "tail set",
    "right tarsal",
    "right paw",
    "left tarsal",
    "left paw",
]

WINDOW_LEN = 30
STRIDE     = 15

# Axis limits for relative coordinates (dx, dy) where origin is tail set
X_LIM = (-300, 300)
Y_LIM = (0, 300)

# Axis limits for normalized coordinates (nx, ny)
X_NORM_LIM = (-1.0, 1.0)
Y_NORM_LIM = (-1.0, 1.0)

INVERT_Y = True           # If True, invert y-axis for plotting
EXPORT_NORMALIZED = True   # If True, output nx,ny plots to process/<keypoint>_nor/


In [6]:
import os, re, glob
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def _norm_name(s: str) -> str:
    return ''.join(ch for ch in s.lower() if ch not in ' _-')

def _resolve_keypoints(all_bodyparts, requested):
    norm2orig = {}
    for bp in all_bodyparts:
        k = _norm_name(bp)
        if k not in norm2orig:
            norm2orig[k] = bp
    resolved, missing = [], []
    for req in requested:
        k = _norm_name(req)
        if k in norm2orig:
            resolved.append(norm2orig[k])
        else:
            missing.append(req)
    if missing:
        raise ValueError(f"Missing keypoints in CSV: {missing}\nAvailable: {all_bodyparts}")
    return resolved

def read_dlc_xy(df: pd.DataFrame, bp: str):
    x = df.xs((bp, 'x'), level=[1,2], axis=1).values.flatten()
    y = df.xs((bp, 'y'), level=[1,2], axis=1).values.flatten()
    return x.astype(np.float32), y.astype(np.float32)

def read_dlc_nxny(df: pd.DataFrame, bp: str):
    coords = set(df.columns.get_level_values(2).unique())
    if not {'nx','ny'}.issubset(coords):
        return None, None  # normalized columns not present
    nx = df.xs((bp, 'nx'), level=[1,2], axis=1).values.flatten()
    ny = df.xs((bp, 'ny'), level=[1,2], axis=1).values.flatten()
    return nx.astype(np.float32), ny.astype(np.float32)

def make_window_starts(T: int, win: int, stride: int):
    if T < win:
        return []
    return list(range(0, T - win + 1, stride))

def ensure_dir(p: Path):
    p.mkdir(parents=True, exist_ok=True)


In [7]:
def export_trajectories(
    in_dir: Path,
    out_root: Path,
    keypoints: list[str],
    window_len: int = 60,
    stride: int = 30,
    x_lim=(-300,300),
    y_lim=(0,300),
    x_norm_lim=(-1.0, 1.0),
    y_norm_lim=(-1.0, 1.0),
    invert_y: bool = False,
    export_normalized: bool = True,
):
    in_dir = Path(in_dir)
    out_root = Path(out_root)
    ensure_dir(out_root)

    csv_paths = sorted(glob.glob(str(in_dir / '*.csv')))
    if not csv_paths:
        print(f"[WARN] No CSV files in {in_dir}")
        return

    for path in csv_paths:
        try:
            df = pd.read_csv(path, header=[0,1,2], index_col=0)
        except Exception as e:
            print(f"[ERROR] cannot read {path}: {e}")
            continue

        bodyparts = list({bp for (_, bp, _) in df.columns})
        try:
            used = _resolve_keypoints(bodyparts, keypoints)
        except ValueError as e:
            print(f"[WARN] {Path(path).name}: {e}")
            continue

        # tail set index
        low = [s.lower() for s in used]
        if 'tail_set' not in low:
            print(f"[WARN] 'tail set' not found in requested keypoints for {Path(path).name}. Skipping.")
            continue
        tidx = low.index('tail_set')
        tail_name = used[tidx]

        # read base (x, y)
        try:
            tx, ty = read_dlc_xy(df, tail_name)
        except Exception as e:
            print(f"[WARN] tail set columns missing in {Path(path).name}: {e}")
            continue
        T = len(tx)
        starts = make_window_starts(T, window_len, stride)
        if not starts:
            print(f"[INFO] too short: {Path(path).name} (T={T})")
            continue

        # For normalized columns availability check once
        have_norm = False
        if export_normalized:
            nx_tail, ny_tail = read_dlc_nxny(df, tail_name)
            have_norm = (nx_tail is not None) and (ny_tail is not None)

        fname = Path(path).stem

        for kpi, kp in enumerate(used):
            if kp.lower() == 'tail_set':
                continue  # no need to output tail itself

            # relative: process/<kp>/
            rel_dir = out_root / kp
            ensure_dir(rel_dir)

            # read kp raw x,y
            try:
                x, y = read_dlc_xy(df, kp)
            except Exception as e:
                print(f"[WARN] {kp} x,y missing in {Path(path).name}: {e}")
                continue

            dx = x - tx
            dy = y - ty

            for s in starts:
                eidx = s + window_len
                segx = dx[s:eidx]
                segy = dy[s:eidx]

                fig, ax = plt.subplots(figsize=(6,6))
                ax.plot(segx, segy)  # line only
                ax.set_xlim(*x_lim)
                ax.set_ylim(*y_lim)
                if invert_y:
                    ax.invert_yaxis()
                ax.set_title(f"{fname} | {kp} | {s}-{eidx}")
                ax.set_xlabel("dx (px)")
                ax.set_ylabel("dy (px)")
                fig.tight_layout()
                out_path = rel_dir / f"{fname}_{_norm_name(kp)}_{s}-{eidx}.png"
                fig.savefig(out_path, dpi=150)
                plt.close(fig)

            # normalized: process/<kp>_nor/
            if export_normalized and have_norm:
                nkx, nky = read_dlc_nxny(df, kp)
                if nkx is None or nky is None:
                    print(f"[WARN] normalized columns missing for {kp} in {Path(path).name}")
                else:
                    nor_dir = out_root / f"{kp}_nor"   # <-- per-keypoint normalized folder (updated)
                    ensure_dir(nor_dir)
                    for s in starts:
                        eidx = s + window_len
                        segx = nkx[s:eidx]
                        segy = nky[s:eidx]

                        fig, ax = plt.subplots(figsize=(6,6))
                        ax.plot(segx, segy)  # line only
                        ax.set_xlim(*x_norm_lim)
                        ax.set_ylim(*y_norm_lim)
                        if invert_y:
                            ax.invert_yaxis()
                        ax.set_title(f"{fname} | {kp} (normalized) | {s}-{eidx}")
                        ax.set_xlabel("nx")
                        ax.set_ylabel("ny")
                        fig.tight_layout()
                        out_path = nor_dir / f"{fname}_{_norm_name(kp)}_{s}-{eidx}_norm.png"
                        fig.savefig(out_path, dpi=150)
                        plt.close(fig)

        print(f"[OK] {Path(path).name} -> done.")


In [8]:
# Run
from argparse import ArgumentParser

def main():
    ap = ArgumentParser()
    ap.add_argument('--input',  type=str, default=IN_DIR)
    ap.add_argument('--output', type=str, default=OUT_DIR)
    ap.add_argument('--keypoints', type=str, default=','.join(KEYPOINTS))
    ap.add_argument('--window-len', type=int, default=WINDOW_LEN)
    ap.add_argument('--stride', type=int, default=STRIDE)
    ap.add_argument('--invert-y', action='store_true', default=INVERT_Y)

    ap.add_argument('--xlim', type=float, nargs=2, default=list(X_LIM))
    ap.add_argument('--ylim', type=float, nargs=2, default=list(Y_LIM))
    ap.add_argument('--xnorm', type=float, nargs=2, default=list(X_NORM_LIM))
    ap.add_argument('--ynorm', type=float, nargs=2, default=list(Y_NORM_LIM))
    ap.add_argument('--export-normalized', action='store_true' if EXPORT_NORMALIZED else 'store_false',
                    default=EXPORT_NORMALIZED)

    args, _ = ap.parse_known_args()

    in_dir  = Path(args.input)
    out_dir = Path(args.output)
    kps = [s.strip() for s in args.keypoints.split(',') if s.strip()]

    export_trajectories(
        in_dir=in_dir,
        out_root=out_dir,
        keypoints=kps,
        window_len=args.window_len,
        stride=args.stride,
        x_lim=tuple(args.xlim),
        y_lim=tuple(args.ylim),
        x_norm_lim=tuple(args.xnorm),
        y_norm_lim=tuple(args.ynorm),
        invert_y=args.invert_y,
        export_normalized=args.export_normalized,
    )

if __name__ == '__main__':
    main()


[OK] normal_100DLC_resnet152_sotuken1Dec17shuffle1_150000_proc.csv -> done.
[OK] normal_101DLC_resnet152_sotuken1Dec17shuffle1_150000_proc.csv -> done.
[OK] normal_102DLC_resnet152_sotuken1Dec17shuffle1_150000_proc.csv -> done.
[OK] normal_103DLC_resnet152_sotuken1Dec17shuffle1_150000_proc.csv -> done.
[OK] normal_104DLC_resnet152_sotuken1Dec17shuffle1_150000_proc.csv -> done.
[OK] normal_105DLC_resnet152_sotuken1Dec17shuffle1_150000_proc.csv -> done.
[OK] normal_106DLC_resnet152_sotuken1Dec17shuffle1_150000_proc.csv -> done.
[OK] normal_107DLC_resnet152_sotuken1Dec17shuffle1_150000_proc.csv -> done.
[OK] normal_108DLC_resnet152_sotuken1Dec17shuffle1_150000_proc.csv -> done.
[OK] normal_109DLC_resnet152_sotuken1Dec17shuffle1_150000_proc.csv -> done.
[OK] normal_10DLC_resnet152_sotuken1Dec17shuffle1_150000_proc.csv -> done.
[OK] normal_110DLC_resnet152_sotuken1Dec17shuffle1_150000_proc.csv -> done.
[OK] normal_111DLC_resnet152_sotuken1Dec17shuffle1_150000_proc.csv -> done.
[OK] normal_1